In [ ]:
# Copyright (c) Meta Platforms, Inc. and affiliates.
# All rights reserved.

# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.

# UniK-QA on WebQSP

In [1]:
import os

# change to your local path
WEBQSP_DIR = os.path.dirname(os.path.realpath('.'))
DATA_DIR = os.path.join(WEBQSP_DIR, "data")
FREEBASE_DIR = os.path.join(DATA_DIR, "freebase_2hop")

In [2]:
# Get entity names from FastRDFStore
# https://github.com/microsoft/FastRDFStore

from struct import *


class BinaryStream:
    def __init__(self, base_stream):
        self.base_stream = base_stream

    def readByte(self):
        return self.base_stream.read(1)

    def readBytes(self, length):
        return self.base_stream.read(length)

    def readChar(self):
        return self.unpack('b')

    def readUChar(self):
        return self.unpack('B')

    def readBool(self):
        return self.unpack('?')

    def readInt16(self):
        return self.unpack('h', 2)

    def readUInt16(self):
        return self.unpack('H', 2)

    def readInt32(self):
        return self.unpack('i', 4)

    def readUInt32(self):
        return self.unpack('I', 4)

    def readInt64(self):
        return self.unpack('q', 8)

    def readUInt64(self):
        return self.unpack('Q', 8)

    def readFloat(self):
        return self.unpack('f', 4)

    def readDouble(self):
        return self.unpack('d', 8)

    def decode_from_7bit(self):
        """
        Decode 7-bit encoded int from str data
        """
        result = 0
        index = 0
        while True:
            byte_value = self.readUChar()
            result |= (byte_value & 0x7f) << (7 * index)
            if byte_value & 0x80 == 0:
                break
            index += 1
        return result

    def readString(self):
        length = self.decode_from_7bit()
        return self.unpack(str(length) + 's', length)

    def writeBytes(self, value):
        self.base_stream.write(value)

    def writeChar(self, value):
        self.pack('c', value)

    def writeUChar(self, value):
        self.pack('C', value)

    def writeBool(self, value):
        self.pack('?', value)

    def writeInt16(self, value):
        self.pack('h', value)

    def writeUInt16(self, value):
        self.pack('H', value)

    def writeInt32(self, value):
        self.pack('i', value)

    def writeUInt32(self, value):
        self.pack('I', value)

    def writeInt64(self, value):
        self.pack('q', value)

    def writeUInt64(self, value):
        self.pack('Q', value)

    def writeFloat(self, value):
        self.pack('f', value)

    def writeDouble(self, value):
        self.pack('d', value)

    def writeString(self, value):
        length = len(value)
        self.writeUInt16(length)
        self.pack(str(length) + 's', value)

    def pack(self, fmt, data):
        return self.writeBytes(pack(fmt, data))

    def unpack(self, fmt, length = 1):
        return unpack(fmt, self.readBytes(length))[0]

In [3]:
import os
ALL_ENTITY_NAME_BIN = os.path.join(DATA_DIR, "FastRDFStore", "data", "namesTable.bin")
entity_names = {}
with open(ALL_ENTITY_NAME_BIN, 'rb') as inf:
    stream = BinaryStream(inf)
    dict_cnt = stream.readInt32()
    print("total entities:", dict_cnt)
    for _ in range(dict_cnt):
        key = stream.readString().decode()
        if key.startswith('m.') or key.startswith('g.'):
            key = '/' + key[0] + '/' + key[2:]
        value = stream.readString().decode()
        entity_names[key] = value

total entities: 51718087


## Utility function for reading graphs

In [4]:
class Relation:
    def __init__(self, line):
        if line is None:
            self.subj = self.rel = self.obj = None
            return
        e1, rel, e2 = line.strip().split(None, 2)
        e1 = self.canonicalize(e1)
        e2 = self.canonicalize(e2)
        self.subj = e1
        self.rel = rel
        self.obj = e2
    
    def __hash__(self):
        return hash((self.subj, self.rel, self.obj))
        
    def _filter_relation(self):
        # same criteria as GraftNet
        relation = self.rel
        if relation == "<fb:common.topic.notable_types>": return False
        domain = relation[4:-1].split(".")[0]
        if domain == "type" or domain == "common": return True
        return False

    def should_ignore(self):
        if self._filter_relation():
            return True
        return False
    
    def canonicalize(self, ent):
        if ent.startswith("<fb:m."):
            return "/m/" + ent[6:-1]
        elif ent.startswith("<fb:g."):
            return "/g/" + ent[6:-1]
        else:
            return ent
    
    def __repr__(self):
        return f"Subj: {self.subj}; Rel: {self.rel}; Obj: {self.obj}"


In [5]:
def read_relations_for_question(qid, ignore_rel=True):
    infname = os.path.join(FREEBASE_DIR, "stagg.neighborhoods", f"{qid}.nxhd")
    if not os.path.exists(infname):
        return None
    relations = []
    edge_map = defaultdict(list)
    with open(infname) as inf:
        for line in inf:
            rel = Relation(line)
            if ignore_rel and rel.should_ignore():
                continue
            relations.append(rel)
            edge_map[rel.subj].append(rel)
            edge_map[rel.obj].append(rel)
    return edge_map, relations

In [6]:
def convert_relation_to_text(relation, entity_names):
    if isinstance(relation, Relation):
        subj, rel, obj = relation.subj, relation.rel, relation.obj
    else:
        subj, rel, obj = relation
    # subject
    if subj in entity_names:
        subj_surface = entity_names[subj]
    else:
        subj_surface = subj
        
    # object
    if obj in entity_names:
        obj_surface = entity_names[obj]
    else:
        obj_surface = obj
            
    # relation
    # e.g. <fb:film.film.other_crew>
    # remove bracket
    rel_surface = rel[4:-1]
    # replace '.' and '_' with ' '
    rel_surface = rel_surface.replace('.', ' ')
    # only keep the last two words
    rel_surface = ' '.join(rel_surface.split(' ')[-2:])
    rel_surface = rel_surface.replace('_', ' ')
    
    return ' '.join([subj_surface, rel_surface, obj_surface, '.'])

In [7]:
# A hyper-relation is also known as a n-ary relation
# Refer to the paper for how we "condense" a n-ary relation into a single sentence.

# if ignore_cvt is True and at least one of subj or obj is unnamed (CVT), return None
def convert_hyper_relation_to_text(relation, entity_names, ignore_cvt=False):
    subj, rels, objs = relation
    # not a hyper-relation
    if not isinstance(rels, list):
        rels = [rels]
    if not isinstance(objs, list):
        objs = [objs]
        
    # subject
    if subj in entity_names:
        subj_surface = entity_names[subj]
    else:
        if ignore_cvt:
            return None
        subj_surface = subj
        
    # object
    obj_surfaces = []
    for obj in objs:
        if obj in entity_names:
            obj_surfaces.append(entity_names[obj])
        else:
            if ignore_cvt and len(obj_surfaces) == 1:
                # don't ignore CVT in hyper-rels
                return None
            obj_surfaces.append(obj)
            
    # relation
    rel_surfaces = []
    for rel in rels:
        # e.g. <fb:film.film.other_crew>
        # remove bracket
        rel_surface = rel[4:-1]
        # replace '.' and '_' with ' '
        rel_surface = rel_surface.replace('.', ' ')
        # only keep the last two words
        rel_surface = ' '.join(rel_surface.split(' ')[-2:])
        rel_surface = rel_surface.replace('_', ' ')
        rel_surfaces.append(rel_surface)
    
    flat_rel = []
    # subj rel1 obj1, and rel2 obj2, ...
    for i, (rs, os) in enumerate(zip(rel_surfaces, obj_surfaces)):
        if i == 0:
            flat_rel.append(' '.join([subj_surface, rs, os]))
        else:
            flat_rel.append(' '.join(['and', rs, os]))
    flat_rel = ' , '.join(flat_rel) + ' .'
    return flat_rel

## Read WebQSP

In [8]:
from collections import defaultdict
import json

# read WebQSP
def get_answers(question):
    """extract unique answers from question parses."""
    answers = set()
    for parse in question["Parses"]:
        for answer in parse["Answers"]:
            answers.add((answer["AnswerArgument"],
                answer["EntityName"]))
    return answers

def get_entities(question):
    """extract oracle entities from question parses."""
    entities = set()
    for parse in question["Parses"]:
        if parse["TopicEntityMid"] is not None:
            entities.add((parse["TopicEntityMid"], parse["TopicEntityName"]))
    return entities

def get_fid(s):
    return '/' + s.replace('.', '/')


webqsp_questions = defaultdict(list)
for split in ['train', 'test']:
    infname = os.path.join(DATA_DIR, "webqsp", f"webqsp_{split}.json")
    data = json.load(open(infname))
    for question in data["Questions"]:
        q_obj = {
            "QuestionId": question["QuestionId"],
            "QuestionText": question["RawQuestion"],
            "ProcessedQuestionText": question["ProcessedQuestion"],
            "OracleEntities": [
                {"freebaseId": get_fid(entity[0]), "text": entity[1]}
                for entity in get_entities(question)
            ],
            "Answers": [
                {"freebaseId": get_fid(answer[0])
                 if answer[0].startswith("m.") or answer[0].startswith("g.") else answer[0],
                 "text": answer[1]}
                for answer in get_answers(question)
            ]
        }
        webqsp_questions[split].append(q_obj)

print(len(webqsp_questions['train']), len(webqsp_questions['test']))

3098 1639


## Prepare DPR Input

Note: We simply dump all relations from each question into a single file, which is sub-optimal.
There're many duplicate relations, but we keep it as is for better replication.
De-duplication can be done to significantly reduce the number of relations DPR need to encode, while not affecting the accuracy at all.

In [9]:
# binary relations

DPR_INPUT_DIR = os.path.join(WEBQSP_DIR, "dpr_inputs")
IDV_REL_DIR = os.path.join(DPR_INPUT_DIR, "individual_relations")
os.makedirs(IDV_REL_DIR, exist_ok=True)

In [10]:
import csv
oufname = os.path.join(IDV_REL_DIR, "all_relations.tsv")

# Note: We simply dump all relations from each question into a single file, which is sub-optimal.
# There're many duplicate relations, but we keep it as is for better replication.
# De-duplication can be done to significantly reduce the number of relations DPR need to encode,
# while not affecting the accuracy at all.
with open(oufname, 'w') as ouf:
    writer = csv.writer(ouf, delimiter='\t')
    writer.writerow(['id', 'text', 'title'])
    
    for split in ['train', 'test']:
        for question in webqsp_questions[split]:
            qid = question['QuestionId']
            print("Processing", qid, "...")
            tup = read_relations_for_question(qid)
            if tup is None:
                continue
            else:
                _, relations = tup
            # write question
            writer.writerow([
                f"{qid}.question",
                question['QuestionText'],
                '',
            ])
            # write relations: each relation is a separate document
            for i, rel in enumerate(relations):
                docid = f"{qid}.relations.{i}"
                text = convert_relation_to_text(rel, entity_names)
                writer.writerow([docid, text, ''])

Processing WebQTrn-0 ...
Processing WebQTrn-1 ...
Processing WebQTrn-3 ...
Processing WebQTrn-4 ...
Processing WebQTrn-5 ...
Processing WebQTrn-6 ...
Processing WebQTrn-7 ...
Processing WebQTrn-8 ...
Processing WebQTrn-9 ...
Processing WebQTrn-11 ...
Processing WebQTrn-12 ...
Processing WebQTrn-13 ...
Processing WebQTrn-14 ...
Processing WebQTrn-15 ...
Processing WebQTrn-17 ...
Processing WebQTrn-18 ...
Processing WebQTrn-20 ...
Processing WebQTrn-21 ...
Processing WebQTrn-22 ...
Processing WebQTrn-23 ...
Processing WebQTrn-24 ...
Processing WebQTrn-25 ...
Processing WebQTrn-26 ...
Processing WebQTrn-30 ...
Processing WebQTrn-31 ...
Processing WebQTrn-32 ...
Processing WebQTrn-33 ...
Processing WebQTrn-34 ...
Processing WebQTrn-35 ...
Processing WebQTrn-36 ...
Processing WebQTrn-37 ...
Processing WebQTrn-38 ...
Processing WebQTrn-40 ...
Processing WebQTrn-42 ...
Processing WebQTrn-44 ...
Processing WebQTrn-45 ...
Processing WebQTrn-46 ...
Processing WebQTrn-47 ...
Processing WebQTrn-49

Processing WebQTrn-379 ...
Processing WebQTrn-380 ...
Processing WebQTrn-381 ...
Processing WebQTrn-382 ...
Processing WebQTrn-383 ...
Processing WebQTrn-385 ...
Processing WebQTrn-386 ...
Processing WebQTrn-387 ...
Processing WebQTrn-388 ...
Processing WebQTrn-389 ...
Processing WebQTrn-390 ...
Processing WebQTrn-391 ...
Processing WebQTrn-393 ...
Processing WebQTrn-395 ...
Processing WebQTrn-397 ...
Processing WebQTrn-398 ...
Processing WebQTrn-399 ...
Processing WebQTrn-400 ...
Processing WebQTrn-401 ...
Processing WebQTrn-402 ...
Processing WebQTrn-403 ...
Processing WebQTrn-404 ...
Processing WebQTrn-405 ...
Processing WebQTrn-406 ...
Processing WebQTrn-407 ...
Processing WebQTrn-408 ...
Processing WebQTrn-409 ...
Processing WebQTrn-410 ...
Processing WebQTrn-411 ...
Processing WebQTrn-412 ...
Processing WebQTrn-413 ...
Processing WebQTrn-414 ...
Processing WebQTrn-415 ...
Processing WebQTrn-416 ...
Processing WebQTrn-417 ...
Processing WebQTrn-418 ...
Processing WebQTrn-420 ...
P

Processing WebQTrn-746 ...
Processing WebQTrn-747 ...
Processing WebQTrn-748 ...
Processing WebQTrn-750 ...
Processing WebQTrn-751 ...
Processing WebQTrn-752 ...
Processing WebQTrn-753 ...
Processing WebQTrn-754 ...
Processing WebQTrn-756 ...
Processing WebQTrn-757 ...
Processing WebQTrn-758 ...
Processing WebQTrn-759 ...
Processing WebQTrn-761 ...
Processing WebQTrn-762 ...
Processing WebQTrn-764 ...
Processing WebQTrn-765 ...
Processing WebQTrn-766 ...
Processing WebQTrn-767 ...
Processing WebQTrn-768 ...
Processing WebQTrn-769 ...
Processing WebQTrn-770 ...
Processing WebQTrn-771 ...
Processing WebQTrn-772 ...
Processing WebQTrn-773 ...
Processing WebQTrn-775 ...
Processing WebQTrn-776 ...
Processing WebQTrn-780 ...
Processing WebQTrn-781 ...
Processing WebQTrn-782 ...
Processing WebQTrn-783 ...
Processing WebQTrn-784 ...
Processing WebQTrn-785 ...
Processing WebQTrn-786 ...
Processing WebQTrn-788 ...
Processing WebQTrn-790 ...
Processing WebQTrn-792 ...
Processing WebQTrn-793 ...
P

Processing WebQTrn-1120 ...
Processing WebQTrn-1121 ...
Processing WebQTrn-1122 ...
Processing WebQTrn-1123 ...
Processing WebQTrn-1124 ...
Processing WebQTrn-1125 ...
Processing WebQTrn-1126 ...
Processing WebQTrn-1127 ...
Processing WebQTrn-1128 ...
Processing WebQTrn-1129 ...
Processing WebQTrn-1130 ...
Processing WebQTrn-1131 ...
Processing WebQTrn-1132 ...
Processing WebQTrn-1133 ...
Processing WebQTrn-1134 ...
Processing WebQTrn-1135 ...
Processing WebQTrn-1136 ...
Processing WebQTrn-1138 ...
Processing WebQTrn-1141 ...
Processing WebQTrn-1142 ...
Processing WebQTrn-1143 ...
Processing WebQTrn-1144 ...
Processing WebQTrn-1148 ...
Processing WebQTrn-1150 ...
Processing WebQTrn-1151 ...
Processing WebQTrn-1152 ...
Processing WebQTrn-1153 ...
Processing WebQTrn-1155 ...
Processing WebQTrn-1156 ...
Processing WebQTrn-1157 ...
Processing WebQTrn-1158 ...
Processing WebQTrn-1160 ...
Processing WebQTrn-1161 ...
Processing WebQTrn-1162 ...
Processing WebQTrn-1163 ...
Processing WebQTrn-1

Processing WebQTrn-1481 ...
Processing WebQTrn-1482 ...
Processing WebQTrn-1484 ...
Processing WebQTrn-1485 ...
Processing WebQTrn-1486 ...
Processing WebQTrn-1487 ...
Processing WebQTrn-1488 ...
Processing WebQTrn-1489 ...
Processing WebQTrn-1492 ...
Processing WebQTrn-1493 ...
Processing WebQTrn-1494 ...
Processing WebQTrn-1495 ...
Processing WebQTrn-1496 ...
Processing WebQTrn-1497 ...
Processing WebQTrn-1498 ...
Processing WebQTrn-1500 ...
Processing WebQTrn-1502 ...
Processing WebQTrn-1503 ...
Processing WebQTrn-1504 ...
Processing WebQTrn-1505 ...
Processing WebQTrn-1506 ...
Processing WebQTrn-1507 ...
Processing WebQTrn-1508 ...
Processing WebQTrn-1511 ...
Processing WebQTrn-1512 ...
Processing WebQTrn-1513 ...
Processing WebQTrn-1515 ...
Processing WebQTrn-1516 ...
Processing WebQTrn-1517 ...
Processing WebQTrn-1519 ...
Processing WebQTrn-1520 ...
Processing WebQTrn-1521 ...
Processing WebQTrn-1522 ...
Processing WebQTrn-1523 ...
Processing WebQTrn-1524 ...
Processing WebQTrn-1

...

Processing WebQTest-616 ...
Processing WebQTest-617 ...
Processing WebQTest-618 ...
Processing WebQTest-619 ...
Processing WebQTest-620 ...
Processing WebQTest-621 ...
Processing WebQTest-622 ...
Processing WebQTest-624 ...
Processing WebQTest-626 ...
Processing WebQTest-627 ...
Processing WebQTest-628 ...
Processing WebQTest-629 ...
Processing WebQTest-630 ...
Processing WebQTest-631 ...
Processing WebQTest-633 ...
Processing WebQTest-634 ...
Processing WebQTest-635 ...
Processing WebQTest-636 ...
Processing WebQTest-637 ...
Processing WebQTest-638 ...
Processing WebQTest-639 ...
Processing WebQTest-640 ...
Processing WebQTest-642 ...
Processing WebQTest-643 ...
Processing WebQTest-644 ...
Processing WebQTest-645 ...
Processing WebQTest-646 ...
Processing WebQTest-648 ...
Processing WebQTest-649 ...
Processing WebQTest-650 ...
Processing WebQTest-651 ...
Processing WebQTest-652 ...
Processing WebQTest-653 ...
Processing WebQTest-654 ...
Processing WebQTest-655 ...
Processing WebQTest-

Processing WebQTest-978 ...
Processing WebQTest-979 ...
Processing WebQTest-980 ...
Processing WebQTest-982 ...
Processing WebQTest-983 ...
Processing WebQTest-984 ...
Processing WebQTest-985 ...
Processing WebQTest-986 ...
Processing WebQTest-987 ...
Processing WebQTest-988 ...
Processing WebQTest-989 ...
Processing WebQTest-990 ...
Processing WebQTest-991 ...
Processing WebQTest-993 ...
Processing WebQTest-994 ...
Processing WebQTest-996 ...
Processing WebQTest-997 ...
Processing WebQTest-998 ...
Processing WebQTest-999 ...
Processing WebQTest-1000 ...
Processing WebQTest-1001 ...
Processing WebQTest-1002 ...
Processing WebQTest-1003 ...
Processing WebQTest-1004 ...
Processing WebQTest-1005 ...
Processing WebQTest-1006 ...
Processing WebQTest-1007 ...
Processing WebQTest-1008 ...
Processing WebQTest-1009 ...
Processing WebQTest-1010 ...
Processing WebQTest-1011 ...
Processing WebQTest-1012 ...
Processing WebQTest-1014 ...
Processing WebQTest-1015 ...
Processing WebQTest-1017 ...
Proc

Processing WebQTest-1330 ...
Processing WebQTest-1331 ...
Processing WebQTest-1332 ...
Processing WebQTest-1333 ...
Processing WebQTest-1334 ...
Processing WebQTest-1335 ...
Processing WebQTest-1336 ...
Processing WebQTest-1337 ...
Processing WebQTest-1339 ...
Processing WebQTest-1340 ...
Processing WebQTest-1341 ...
Processing WebQTest-1342 ...
Processing WebQTest-1343 ...
Processing WebQTest-1344 ...
Processing WebQTest-1348 ...
Processing WebQTest-1350 ...
Processing WebQTest-1351 ...
Processing WebQTest-1353 ...
Processing WebQTest-1354 ...
Processing WebQTest-1355 ...
Processing WebQTest-1356 ...
Processing WebQTest-1357 ...
Processing WebQTest-1358 ...
Processing WebQTest-1359 ...
Processing WebQTest-1361 ...
Processing WebQTest-1362 ...
Processing WebQTest-1363 ...
Processing WebQTest-1364 ...
Processing WebQTest-1365 ...
Processing WebQTest-1366 ...
Processing WebQTest-1367 ...
Processing WebQTest-1368 ...
Processing WebQTest-1369 ...
Processing WebQTest-1370 ...
Processing Web

Processing WebQTest-1682 ...
Processing WebQTest-1683 ...
Processing WebQTest-1684 ...
Processing WebQTest-1685 ...
Processing WebQTest-1686 ...
Processing WebQTest-1687 ...
Processing WebQTest-1688 ...
Processing WebQTest-1690 ...
Processing WebQTest-1691 ...
Processing WebQTest-1694 ...
Processing WebQTest-1695 ...
Processing WebQTest-1696 ...
Processing WebQTest-1698 ...
Processing WebQTest-1699 ...
Processing WebQTest-1700 ...
Processing WebQTest-1701 ...
Processing WebQTest-1702 ...
Processing WebQTest-1703 ...
Processing WebQTest-1705 ...
Processing WebQTest-1706 ...
Processing WebQTest-1707 ...
Processing WebQTest-1708 ...
Processing WebQTest-1710 ...
Processing WebQTest-1711 ...
Processing WebQTest-1712 ...
Processing WebQTest-1713 ...
Processing WebQTest-1714 ...
Processing WebQTest-1715 ...
Processing WebQTest-1716 ...
Processing WebQTest-1719 ...
Processing WebQTest-1721 ...
Processing WebQTest-1723 ...
Processing WebQTest-1724 ...
Processing WebQTest-1725 ...
Processing Web

Processing WebQTest-2029 ...
Processing WebQTest-2030 ...
Processing WebQTest-2031 ...


In [11]:
# condensed relations are n-ary relation broken into n binary relations with CVT entities removed
CONDENSED_KB_DIR = os.path.join(FREEBASE_DIR, "condensed.stagg.neighborhoods", "condensed_edges_only")

In [12]:
def read_condensed_relations_for_question(qid):
    infname = os.path.join(CONDENSED_KB_DIR, f"{qid}.nxhd")
    if not os.path.exists(infname):
        return None
    relations = []
    with open(infname) as inf:
        for line in inf:
            docid, subj, rel, obj = line.strip().split('\t')
            relations.append((docid, (subj, rel, obj)))
    return relations

In [13]:
# A condensed hyper-relation is a n-ary relation with CVT entities removed
CONDENSED_HYPER_KB_DIR = os.path.join(FREEBASE_DIR, "condensed_hyper.stagg.neighborhoods", "condensed_hyper_relations_only")

In [14]:
def read_condensed_hyper_relations_for_question(qid):
    infname = os.path.join(CONDENSED_HYPER_KB_DIR, f"{qid}.nxhd")
    if not os.path.exists(infname):
        return None
    relations = []
    with open(infname) as inf:
        for line in inf:
            parts = line.strip().split('\t')
            docid = parts[0]
            subj = parts[1]
            parts = parts[2:]
            preds = parts[::2]
            objs = parts[1::2]
            assert len(preds) == len(objs)
            relations.append((docid, (subj, preds, objs)))
    return relations

In [15]:
# condense relations
COND_REL_DIR = os.path.join(DPR_INPUT_DIR, "condensed_relations")
os.makedirs(COND_REL_DIR, exist_ok=True)

In [16]:
import csv
oufname = os.path.join(COND_REL_DIR, "all_condensed_relations.tsv")

with open(oufname, 'w') as ouf:
    writer = csv.writer(ouf, delimiter='\t')
    writer.writerow(['id', 'text', 'title'])
    
    for split in ['train', 'test']:
        for question in webqsp_questions[split]:
            qid = question['QuestionId']
            print("Processing", qid, "...")
            relations = read_condensed_relations_for_question(qid)
            if relations is None:
                print("No condensed relations found for", qid)
                continue
            # write condensed relations: each relation is a separate document
            for i, tup in enumerate(relations):
                docid1 = f"{qid}.condensed_relations.{i}"
                docid, rel = tup
                assert docid == docid1
                text = convert_relation_to_text(rel, entity_names)
                writer.writerow([docid, text, ''])    

Processing WebQTrn-0 ...
Processing WebQTrn-1 ...
Processing WebQTrn-3 ...
Processing WebQTrn-4 ...
Processing WebQTrn-5 ...
Processing WebQTrn-6 ...
Processing WebQTrn-7 ...
Processing WebQTrn-8 ...
Processing WebQTrn-9 ...
Processing WebQTrn-11 ...
Processing WebQTrn-12 ...
Processing WebQTrn-13 ...
Processing WebQTrn-14 ...
Processing WebQTrn-15 ...
Processing WebQTrn-17 ...
Processing WebQTrn-18 ...
Processing WebQTrn-20 ...
Processing WebQTrn-21 ...
Processing WebQTrn-22 ...
Processing WebQTrn-23 ...
Processing WebQTrn-24 ...
Processing WebQTrn-25 ...
Processing WebQTrn-26 ...
Processing WebQTrn-30 ...
Processing WebQTrn-31 ...
Processing WebQTrn-32 ...
Processing WebQTrn-33 ...
Processing WebQTrn-34 ...
Processing WebQTrn-35 ...
Processing WebQTrn-36 ...
Processing WebQTrn-37 ...
Processing WebQTrn-38 ...
Processing WebQTrn-40 ...
Processing WebQTrn-42 ...
Processing WebQTrn-44 ...
Processing WebQTrn-45 ...
Processing WebQTrn-46 ...
Processing WebQTrn-47 ...
Processing WebQTrn-49

Processing WebQTrn-387 ...
Processing WebQTrn-388 ...
Processing WebQTrn-389 ...
Processing WebQTrn-390 ...
Processing WebQTrn-391 ...
Processing WebQTrn-393 ...
Processing WebQTrn-395 ...
Processing WebQTrn-397 ...
Processing WebQTrn-398 ...
Processing WebQTrn-399 ...
Processing WebQTrn-400 ...
Processing WebQTrn-401 ...
Processing WebQTrn-402 ...
Processing WebQTrn-403 ...
Processing WebQTrn-404 ...
Processing WebQTrn-405 ...
Processing WebQTrn-406 ...
Processing WebQTrn-407 ...
Processing WebQTrn-408 ...
Processing WebQTrn-409 ...
Processing WebQTrn-410 ...
Processing WebQTrn-411 ...
Processing WebQTrn-412 ...
Processing WebQTrn-413 ...
Processing WebQTrn-414 ...
Processing WebQTrn-415 ...
Processing WebQTrn-416 ...
Processing WebQTrn-417 ...
Processing WebQTrn-418 ...
Processing WebQTrn-420 ...
Processing WebQTrn-421 ...
Processing WebQTrn-422 ...
Processing WebQTrn-423 ...
Processing WebQTrn-424 ...
Processing WebQTrn-426 ...
Processing WebQTrn-427 ...
Processing WebQTrn-428 ...
P

Processing WebQTrn-782 ...
Processing WebQTrn-783 ...
Processing WebQTrn-784 ...
Processing WebQTrn-785 ...
Processing WebQTrn-786 ...
Processing WebQTrn-788 ...
Processing WebQTrn-790 ...
Processing WebQTrn-792 ...
Processing WebQTrn-793 ...
Processing WebQTrn-794 ...
Processing WebQTrn-795 ...
Processing WebQTrn-796 ...
Processing WebQTrn-797 ...
Processing WebQTrn-800 ...
Processing WebQTrn-801 ...
Processing WebQTrn-804 ...
Processing WebQTrn-806 ...
Processing WebQTrn-808 ...
Processing WebQTrn-809 ...
Processing WebQTrn-810 ...
Processing WebQTrn-811 ...
Processing WebQTrn-812 ...
Processing WebQTrn-813 ...
Processing WebQTrn-815 ...
Processing WebQTrn-816 ...
Processing WebQTrn-817 ...
Processing WebQTrn-819 ...
Processing WebQTrn-821 ...
Processing WebQTrn-822 ...
Processing WebQTrn-823 ...
Processing WebQTrn-824 ...
Processing WebQTrn-825 ...
Processing WebQTrn-826 ...
Processing WebQTrn-827 ...
Processing WebQTrn-828 ...
Processing WebQTrn-829 ...
Processing WebQTrn-831 ...
P

Processing WebQTrn-1181 ...
Processing WebQTrn-1182 ...
Processing WebQTrn-1183 ...
Processing WebQTrn-1184 ...
Processing WebQTrn-1185 ...
Processing WebQTrn-1186 ...
Processing WebQTrn-1187 ...
Processing WebQTrn-1188 ...
Processing WebQTrn-1191 ...
Processing WebQTrn-1192 ...
Processing WebQTrn-1193 ...
Processing WebQTrn-1194 ...
Processing WebQTrn-1198 ...
Processing WebQTrn-1199 ...
Processing WebQTrn-1200 ...
Processing WebQTrn-1201 ...
Processing WebQTrn-1202 ...
Processing WebQTrn-1203 ...
Processing WebQTrn-1204 ...
Processing WebQTrn-1205 ...
Processing WebQTrn-1206 ...
Processing WebQTrn-1207 ...
Processing WebQTrn-1208 ...
Processing WebQTrn-1209 ...
Processing WebQTrn-1210 ...
Processing WebQTrn-1211 ...
Processing WebQTrn-1212 ...
Processing WebQTrn-1213 ...
Processing WebQTrn-1214 ...
Processing WebQTrn-1215 ...
Processing WebQTrn-1216 ...
Processing WebQTrn-1217 ...
Processing WebQTrn-1218 ...
Processing WebQTrn-1219 ...
Processing WebQTrn-1220 ...
Processing WebQTrn-1

Processing WebQTrn-1586 ...
Processing WebQTrn-1587 ...
Processing WebQTrn-1589 ...
Processing WebQTrn-1591 ...
Processing WebQTrn-1592 ...
Processing WebQTrn-1594 ...
Processing WebQTrn-1595 ...
Processing WebQTrn-1596 ...
Processing WebQTrn-1597 ...
Processing WebQTrn-1598 ...
Processing WebQTrn-1599 ...
Processing WebQTrn-1601 ...
Processing WebQTrn-1602 ...
Processing WebQTrn-1603 ...
Processing WebQTrn-1604 ...
Processing WebQTrn-1605 ...
Processing WebQTrn-1606 ...
Processing WebQTrn-1607 ...
Processing WebQTrn-1608 ...
Processing WebQTrn-1610 ...
Processing WebQTrn-1611 ...
Processing WebQTrn-1612 ...
Processing WebQTrn-1613 ...
Processing WebQTrn-1614 ...
Processing WebQTrn-1615 ...
Processing WebQTrn-1616 ...
Processing WebQTrn-1617 ...
Processing WebQTrn-1618 ...
Processing WebQTrn-1620 ...
Processing WebQTrn-1621 ...
Processing WebQTrn-1622 ...
Processing WebQTrn-1623 ...
Processing WebQTrn-1624 ...
Processing WebQTrn-1625 ...
Processing WebQTrn-1626 ...
Processing WebQTrn-1

...

Processing WebQTest-71 ...
Processing WebQTest-72 ...
Processing WebQTest-73 ...
Processing WebQTest-74 ...
Processing WebQTest-75 ...
Processing WebQTest-76 ...
Processing WebQTest-77 ...
Processing WebQTest-78 ...
Processing WebQTest-79 ...
Processing WebQTest-80 ...
Processing WebQTest-82 ...
Processing WebQTest-83 ...
Processing WebQTest-84 ...
Processing WebQTest-85 ...
Processing WebQTest-86 ...
Processing WebQTest-87 ...
Processing WebQTest-88 ...
Processing WebQTest-89 ...
Processing WebQTest-90 ...
Processing WebQTest-91 ...
Processing WebQTest-93 ...
Processing WebQTest-94 ...
Processing WebQTest-95 ...
Processing WebQTest-96 ...
Processing WebQTest-97 ...
Processing WebQTest-98 ...
Processing WebQTest-99 ...
Processing WebQTest-100 ...
Processing WebQTest-102 ...
Processing WebQTest-103 ...
Processing WebQTest-104 ...
Processing WebQTest-105 ...
Processing WebQTest-106 ...
Processing WebQTest-107 ...
Processing WebQTest-108 ...
Processing WebQTest-109 ...
Processing WebQTest

Processing WebQTest-454 ...
Processing WebQTest-455 ...
Processing WebQTest-456 ...
Processing WebQTest-459 ...
Processing WebQTest-461 ...
Processing WebQTest-462 ...
Processing WebQTest-463 ...
Processing WebQTest-464 ...
Processing WebQTest-465 ...
Processing WebQTest-466 ...
Processing WebQTest-467 ...
Processing WebQTest-468 ...
Processing WebQTest-469 ...
Processing WebQTest-470 ...
Processing WebQTest-472 ...
Processing WebQTest-473 ...
Processing WebQTest-474 ...
Processing WebQTest-475 ...
Processing WebQTest-477 ...
Processing WebQTest-479 ...
Processing WebQTest-480 ...
Processing WebQTest-481 ...
Processing WebQTest-482 ...
Processing WebQTest-484 ...
Processing WebQTest-487 ...
Processing WebQTest-488 ...
Processing WebQTest-489 ...
Processing WebQTest-490 ...
Processing WebQTest-491 ...
Processing WebQTest-492 ...
Processing WebQTest-493 ...
Processing WebQTest-497 ...
Processing WebQTest-500 ...
Processing WebQTest-501 ...
Processing WebQTest-502 ...
Processing WebQTest-

Processing WebQTest-833 ...
Processing WebQTest-834 ...
Processing WebQTest-836 ...
Processing WebQTest-837 ...
Processing WebQTest-838 ...
Processing WebQTest-841 ...
Processing WebQTest-843 ...
Processing WebQTest-845 ...
Processing WebQTest-846 ...
Processing WebQTest-847 ...
Processing WebQTest-848 ...
Processing WebQTest-849 ...
Processing WebQTest-850 ...
Processing WebQTest-851 ...
Processing WebQTest-852 ...
Processing WebQTest-855 ...
Processing WebQTest-856 ...
Processing WebQTest-857 ...
Processing WebQTest-859 ...
Processing WebQTest-861 ...
Processing WebQTest-862 ...
Processing WebQTest-863 ...
Processing WebQTest-865 ...
Processing WebQTest-866 ...
Processing WebQTest-867 ...
Processing WebQTest-868 ...
Processing WebQTest-869 ...
Processing WebQTest-870 ...
Processing WebQTest-872 ...
Processing WebQTest-873 ...
Processing WebQTest-874 ...
Processing WebQTest-875 ...
Processing WebQTest-876 ...
Processing WebQTest-877 ...
Processing WebQTest-878 ...
Processing WebQTest-

Processing WebQTest-1263 ...
Processing WebQTest-1264 ...
Processing WebQTest-1265 ...
Processing WebQTest-1266 ...
Processing WebQTest-1267 ...
Processing WebQTest-1268 ...
Processing WebQTest-1269 ...
Processing WebQTest-1270 ...
Processing WebQTest-1271 ...
Processing WebQTest-1272 ...
Processing WebQTest-1274 ...
Processing WebQTest-1275 ...
Processing WebQTest-1276 ...
Processing WebQTest-1277 ...
Processing WebQTest-1278 ...
Processing WebQTest-1279 ...
Processing WebQTest-1280 ...
Processing WebQTest-1281 ...
Processing WebQTest-1282 ...
Processing WebQTest-1284 ...
Processing WebQTest-1285 ...
Processing WebQTest-1287 ...
Processing WebQTest-1289 ...
Processing WebQTest-1291 ...
Processing WebQTest-1292 ...
Processing WebQTest-1293 ...
Processing WebQTest-1294 ...
Processing WebQTest-1295 ...
Processing WebQTest-1296 ...
Processing WebQTest-1297 ...
Processing WebQTest-1298 ...
Processing WebQTest-1300 ...
Processing WebQTest-1301 ...
Processing WebQTest-1303 ...
Processing Web

Processing WebQTest-1680 ...
Processing WebQTest-1682 ...
Processing WebQTest-1683 ...
Processing WebQTest-1684 ...
Processing WebQTest-1685 ...
Processing WebQTest-1686 ...
Processing WebQTest-1687 ...
Processing WebQTest-1688 ...
Processing WebQTest-1690 ...
Processing WebQTest-1691 ...
Processing WebQTest-1694 ...
Processing WebQTest-1695 ...
Processing WebQTest-1696 ...
Processing WebQTest-1698 ...
Processing WebQTest-1699 ...
Processing WebQTest-1700 ...
Processing WebQTest-1701 ...
Processing WebQTest-1702 ...
Processing WebQTest-1703 ...
Processing WebQTest-1705 ...
Processing WebQTest-1706 ...
Processing WebQTest-1707 ...
Processing WebQTest-1708 ...
Processing WebQTest-1710 ...
Processing WebQTest-1711 ...
Processing WebQTest-1712 ...
Processing WebQTest-1713 ...
Processing WebQTest-1714 ...
Processing WebQTest-1715 ...
Processing WebQTest-1716 ...
No condensed relations found for WebQTest-1716
Processing WebQTest-1719 ...
Processing WebQTest-1721 ...
Processing WebQTest-1723 

In [17]:
# condensed hyper relations

import csv
COND_HYPERREL_DIR = os.path.join(DPR_INPUT_DIR, "condensed_hyper_relations")
os.makedirs(COND_HYPERREL_DIR, exist_ok=True)

In [18]:
oufname = os.path.join(COND_HYPERREL_DIR, "all_condensed_hyper_relations.tsv")

with open(oufname, 'w') as ouf:
    writer = csv.writer(ouf, delimiter='\t')
    writer.writerow(['id', 'text', 'title'])
    
    for split in ['train', 'test']:
        for question in webqsp_questions[split]:
            qid = question['QuestionId']
            print("Processing", qid, "...")
            relations = read_condensed_hyper_relations_for_question(qid)
            if relations is None:
                print("No condensed relations found for", qid)
                continue
            # write condensed relations
            for docid, rel in relations:
                text = convert_hyper_relation_to_text(rel, entity_names)
                writer.writerow([docid, text, ''])

Processing WebQTrn-0 ...
Processing WebQTrn-1 ...
Processing WebQTrn-3 ...
Processing WebQTrn-4 ...
Processing WebQTrn-5 ...
Processing WebQTrn-6 ...
Processing WebQTrn-7 ...
Processing WebQTrn-8 ...
Processing WebQTrn-9 ...
Processing WebQTrn-11 ...
Processing WebQTrn-12 ...
Processing WebQTrn-13 ...
Processing WebQTrn-14 ...
Processing WebQTrn-15 ...
Processing WebQTrn-17 ...
Processing WebQTrn-18 ...
Processing WebQTrn-20 ...
Processing WebQTrn-21 ...
Processing WebQTrn-22 ...
Processing WebQTrn-23 ...
Processing WebQTrn-24 ...
Processing WebQTrn-25 ...
Processing WebQTrn-26 ...
Processing WebQTrn-30 ...
Processing WebQTrn-31 ...
Processing WebQTrn-32 ...
Processing WebQTrn-33 ...
Processing WebQTrn-34 ...
Processing WebQTrn-35 ...
Processing WebQTrn-36 ...
Processing WebQTrn-37 ...
Processing WebQTrn-38 ...
Processing WebQTrn-40 ...
Processing WebQTrn-42 ...
Processing WebQTrn-44 ...
Processing WebQTrn-45 ...
Processing WebQTrn-46 ...
Processing WebQTrn-47 ...
Processing WebQTrn-49

Processing WebQTrn-399 ...
Processing WebQTrn-400 ...
Processing WebQTrn-401 ...
Processing WebQTrn-402 ...
Processing WebQTrn-403 ...
Processing WebQTrn-404 ...
Processing WebQTrn-405 ...
Processing WebQTrn-406 ...
Processing WebQTrn-407 ...
Processing WebQTrn-408 ...
Processing WebQTrn-409 ...
Processing WebQTrn-410 ...
Processing WebQTrn-411 ...
Processing WebQTrn-412 ...
Processing WebQTrn-413 ...
Processing WebQTrn-414 ...
Processing WebQTrn-415 ...
Processing WebQTrn-416 ...
Processing WebQTrn-417 ...
Processing WebQTrn-418 ...
Processing WebQTrn-420 ...
Processing WebQTrn-421 ...
Processing WebQTrn-422 ...
Processing WebQTrn-423 ...
Processing WebQTrn-424 ...
Processing WebQTrn-426 ...
Processing WebQTrn-427 ...
Processing WebQTrn-428 ...
Processing WebQTrn-429 ...
Processing WebQTrn-430 ...
Processing WebQTrn-431 ...
Processing WebQTrn-432 ...
Processing WebQTrn-434 ...
Processing WebQTrn-436 ...
Processing WebQTrn-437 ...
Processing WebQTrn-440 ...
Processing WebQTrn-441 ...
P

Processing WebQTrn-793 ...
Processing WebQTrn-794 ...
Processing WebQTrn-795 ...
Processing WebQTrn-796 ...
Processing WebQTrn-797 ...
Processing WebQTrn-800 ...
Processing WebQTrn-801 ...
Processing WebQTrn-804 ...
Processing WebQTrn-806 ...
Processing WebQTrn-808 ...
Processing WebQTrn-809 ...
Processing WebQTrn-810 ...
Processing WebQTrn-811 ...
Processing WebQTrn-812 ...
Processing WebQTrn-813 ...
Processing WebQTrn-815 ...
Processing WebQTrn-816 ...
Processing WebQTrn-817 ...
Processing WebQTrn-819 ...
Processing WebQTrn-821 ...
Processing WebQTrn-822 ...
Processing WebQTrn-823 ...
Processing WebQTrn-824 ...
Processing WebQTrn-825 ...
Processing WebQTrn-826 ...
Processing WebQTrn-827 ...
Processing WebQTrn-828 ...
Processing WebQTrn-829 ...
Processing WebQTrn-831 ...
Processing WebQTrn-832 ...
Processing WebQTrn-833 ...
Processing WebQTrn-834 ...
Processing WebQTrn-836 ...
Processing WebQTrn-837 ...
Processing WebQTrn-838 ...
Processing WebQTrn-839 ...
Processing WebQTrn-840 ...
P

Processing WebQTrn-1169 ...
Processing WebQTrn-1170 ...
Processing WebQTrn-1172 ...
Processing WebQTrn-1173 ...
Processing WebQTrn-1174 ...
Processing WebQTrn-1175 ...
Processing WebQTrn-1176 ...
Processing WebQTrn-1177 ...
Processing WebQTrn-1178 ...
Processing WebQTrn-1179 ...
Processing WebQTrn-1180 ...
Processing WebQTrn-1181 ...
Processing WebQTrn-1182 ...
Processing WebQTrn-1183 ...
Processing WebQTrn-1184 ...
Processing WebQTrn-1185 ...
Processing WebQTrn-1186 ...
Processing WebQTrn-1187 ...
Processing WebQTrn-1188 ...
Processing WebQTrn-1191 ...
Processing WebQTrn-1192 ...
Processing WebQTrn-1193 ...
Processing WebQTrn-1194 ...
Processing WebQTrn-1198 ...
Processing WebQTrn-1199 ...
Processing WebQTrn-1200 ...
Processing WebQTrn-1201 ...
Processing WebQTrn-1202 ...
Processing WebQTrn-1203 ...
Processing WebQTrn-1204 ...
Processing WebQTrn-1205 ...
Processing WebQTrn-1206 ...
Processing WebQTrn-1207 ...
Processing WebQTrn-1208 ...
Processing WebQTrn-1209 ...
Processing WebQTrn-1

Processing WebQTrn-1525 ...
Processing WebQTrn-1527 ...
Processing WebQTrn-1528 ...
Processing WebQTrn-1529 ...
Processing WebQTrn-1530 ...
Processing WebQTrn-1531 ...
Processing WebQTrn-1532 ...
Processing WebQTrn-1534 ...
Processing WebQTrn-1535 ...
Processing WebQTrn-1536 ...
Processing WebQTrn-1537 ...
Processing WebQTrn-1538 ...
Processing WebQTrn-1539 ...
Processing WebQTrn-1540 ...
Processing WebQTrn-1541 ...
Processing WebQTrn-1542 ...
Processing WebQTrn-1545 ...
Processing WebQTrn-1546 ...
Processing WebQTrn-1548 ...
Processing WebQTrn-1549 ...
Processing WebQTrn-1550 ...
Processing WebQTrn-1551 ...
Processing WebQTrn-1552 ...
Processing WebQTrn-1553 ...
Processing WebQTrn-1554 ...
Processing WebQTrn-1555 ...
Processing WebQTrn-1556 ...
Processing WebQTrn-1557 ...
Processing WebQTrn-1559 ...
Processing WebQTrn-1560 ...
Processing WebQTrn-1561 ...
Processing WebQTrn-1562 ...
Processing WebQTrn-1563 ...
Processing WebQTrn-1564 ...
Processing WebQTrn-1565 ...
Processing WebQTrn-1

...

Processing WebQTest-428 ...
Processing WebQTest-430 ...
Processing WebQTest-431 ...
Processing WebQTest-432 ...
Processing WebQTest-433 ...
Processing WebQTest-434 ...
Processing WebQTest-436 ...
Processing WebQTest-437 ...
Processing WebQTest-438 ...
Processing WebQTest-439 ...
Processing WebQTest-440 ...
Processing WebQTest-441 ...
Processing WebQTest-442 ...
Processing WebQTest-443 ...
Processing WebQTest-446 ...
Processing WebQTest-447 ...
Processing WebQTest-448 ...
Processing WebQTest-449 ...
Processing WebQTest-450 ...
Processing WebQTest-451 ...
Processing WebQTest-453 ...
Processing WebQTest-454 ...
Processing WebQTest-455 ...
Processing WebQTest-456 ...
Processing WebQTest-459 ...
Processing WebQTest-461 ...
Processing WebQTest-462 ...
Processing WebQTest-463 ...
Processing WebQTest-464 ...
Processing WebQTest-465 ...
Processing WebQTest-466 ...
Processing WebQTest-467 ...
Processing WebQTest-468 ...
Processing WebQTest-469 ...
Processing WebQTest-470 ...
Processing WebQTest-

Processing WebQTest-801 ...
Processing WebQTest-802 ...
Processing WebQTest-805 ...
Processing WebQTest-809 ...
Processing WebQTest-810 ...
Processing WebQTest-811 ...
Processing WebQTest-812 ...
Processing WebQTest-813 ...
Processing WebQTest-814 ...
Processing WebQTest-818 ...
Processing WebQTest-819 ...
Processing WebQTest-820 ...
Processing WebQTest-821 ...
Processing WebQTest-822 ...
Processing WebQTest-823 ...
Processing WebQTest-825 ...
Processing WebQTest-827 ...
Processing WebQTest-828 ...
Processing WebQTest-829 ...
Processing WebQTest-830 ...
Processing WebQTest-831 ...
Processing WebQTest-832 ...
Processing WebQTest-833 ...
Processing WebQTest-834 ...
Processing WebQTest-836 ...
Processing WebQTest-837 ...
Processing WebQTest-838 ...
Processing WebQTest-841 ...
Processing WebQTest-843 ...
Processing WebQTest-845 ...
Processing WebQTest-846 ...
Processing WebQTest-847 ...
Processing WebQTest-848 ...
Processing WebQTest-849 ...
Processing WebQTest-850 ...
Processing WebQTest-

Processing WebQTest-1225 ...
Processing WebQTest-1226 ...
Processing WebQTest-1227 ...
Processing WebQTest-1228 ...
Processing WebQTest-1229 ...
Processing WebQTest-1230 ...
Processing WebQTest-1231 ...
Processing WebQTest-1232 ...
Processing WebQTest-1233 ...
Processing WebQTest-1234 ...
Processing WebQTest-1235 ...
Processing WebQTest-1237 ...
Processing WebQTest-1238 ...
Processing WebQTest-1239 ...
Processing WebQTest-1240 ...
Processing WebQTest-1241 ...
Processing WebQTest-1242 ...
Processing WebQTest-1244 ...
Processing WebQTest-1245 ...
Processing WebQTest-1246 ...
Processing WebQTest-1247 ...
Processing WebQTest-1248 ...
Processing WebQTest-1249 ...
Processing WebQTest-1250 ...
Processing WebQTest-1251 ...
Processing WebQTest-1252 ...
Processing WebQTest-1254 ...
Processing WebQTest-1255 ...
Processing WebQTest-1256 ...
Processing WebQTest-1257 ...
Processing WebQTest-1259 ...
Processing WebQTest-1260 ...
Processing WebQTest-1261 ...
Processing WebQTest-1262 ...
Processing Web

Processing WebQTest-1582 ...
Processing WebQTest-1583 ...
Processing WebQTest-1584 ...
Processing WebQTest-1585 ...
Processing WebQTest-1586 ...
Processing WebQTest-1587 ...
Processing WebQTest-1589 ...
Processing WebQTest-1590 ...
Processing WebQTest-1591 ...
Processing WebQTest-1592 ...
Processing WebQTest-1593 ...
Processing WebQTest-1595 ...
Processing WebQTest-1596 ...
Processing WebQTest-1597 ...
Processing WebQTest-1598 ...
Processing WebQTest-1599 ...
Processing WebQTest-1600 ...
Processing WebQTest-1603 ...
Processing WebQTest-1604 ...
Processing WebQTest-1607 ...
Processing WebQTest-1608 ...
Processing WebQTest-1609 ...
Processing WebQTest-1610 ...
Processing WebQTest-1611 ...
Processing WebQTest-1612 ...
Processing WebQTest-1613 ...
Processing WebQTest-1614 ...
Processing WebQTest-1615 ...
Processing WebQTest-1616 ...
Processing WebQTest-1617 ...
Processing WebQTest-1618 ...
Processing WebQTest-1619 ...
Processing WebQTest-1620 ...
Processing WebQTest-1621 ...
Processing Web

Processing WebQTest-1961 ...
Processing WebQTest-1962 ...
Processing WebQTest-1963 ...
Processing WebQTest-1964 ...
Processing WebQTest-1965 ...
Processing WebQTest-1966 ...
Processing WebQTest-1967 ...
Processing WebQTest-1968 ...
Processing WebQTest-1969 ...
Processing WebQTest-1971 ...
Processing WebQTest-1972 ...
Processing WebQTest-1975 ...
Processing WebQTest-1976 ...
Processing WebQTest-1977 ...
Processing WebQTest-1978 ...
Processing WebQTest-1979 ...
Processing WebQTest-1981 ...
Processing WebQTest-1982 ...
Processing WebQTest-1985 ...
Processing WebQTest-1986 ...
Processing WebQTest-1987 ...
Processing WebQTest-1988 ...
Processing WebQTest-1990 ...
Processing WebQTest-1991 ...
Processing WebQTest-1992 ...
Processing WebQTest-1993 ...
Processing WebQTest-1994 ...
Processing WebQTest-1996 ...
Processing WebQTest-1997 ...
Processing WebQTest-1998 ...
Processing WebQTest-1999 ...
Processing WebQTest-2000 ...
Processing WebQTest-2001 ...
Processing WebQTest-2002 ...
Processing Web

## Run DPR Inference

We can now use the official DPR code and checkpoint (https://github.com/facebookresearch/DPR) to generate the query and passage embeddings using the DPR input generated above.

For reproducibility, we release the pre-computed DPR embeddings, which can be downloaded [here](https://dl.fbaipublicfiles.com/UniK-QA/dpr_predictions.tar.xz).

```bash
mkdir models & cd models
wget https://dl.fbaipublicfiles.com/UniK-QA/dpr_predictions.tar.xz
tar -xvf dpr_predictions.tar.xz
cd ..
```

We provide the original embeddings we used in our paper for reproducing our results, but there are at least two optimizations that can be done: i) We used the NQ-single DPR checkpoint, but the DPR-multiset checkpoint may offer higher performance; and ii) As mentioned in a comment above, we did not deduplicate the DPR input, which makes the DPR embeddings extremely large (700GB). The size can be significantly reduced with deduplication, which will also increase the speed.

(Alternatively, you can use DPR to generate the embeddings using the DPR input files produced in the previous steps. If the embeddings are too large, the aforementioned deduplication can be implemented by modifying this script to significantly reduce the file size and inference time.)

## DPR Output Pruning for the subgraph of each question

Extract top 2000 relations (binary relations + n-ary relations) for each question using DPR scores.

In [19]:
# read DPR embeddings for condensed relations
import csv
import pickle
from collections import defaultdict

COND_DPR_OUT_DIR = os.path.join(WEBQSP_DIR, "models", "dpr_predictions", "webqsp_condensed_relations")

# read all condensed relations       
NUM_SHARDS_CONDENSED = 16
condensed_rels_vectors = defaultdict(list)
for shard in range(NUM_SHARDS_CONDENSED):
    condensed_dpr_output_file = os.path.join(COND_DPR_OUT_DIR, f"all_condensed_relations.dpr_out_{shard}")
    with open(condensed_dpr_output_file, 'rb') as inf:
        dpr_output = pickle.load(inf)
        for docid, vec in dpr_output:
            parts = docid.split('.')
            qid, dtype = parts[0], parts[1]
            condensed_rels_vectors[qid].append((docid, vec))

In [20]:
# read DPR embeddings for condensed hyper-relations
import csv
import pickle
from collections import defaultdict

COND_HYPER_DPR_OUT_DIR = os.path.join(WEBQSP_DIR, "models", "dpr_predictions", "webqsp_condensed_hyper_relations")

# read all condensed relations       
NUM_SHARDS_CONDENSED_HYPER = 16
condensed_hyper_rels_vectors = defaultdict(list)
for shard in range(NUM_SHARDS_CONDENSED_HYPER):
    condensed_dpr_output_file = os.path.join(COND_HYPER_DPR_OUT_DIR, f"all_condensed_hyper_relations.dpr_out_{shard}")
    with open(condensed_dpr_output_file, 'rb') as inf:
        dpr_output = pickle.load(inf)
        for docid, vec in dpr_output:
            parts = docid.split('.')
            qid, dtype = parts[0], parts[1]
            condensed_hyper_rels_vectors[qid].append((docid, vec))

In [21]:
SUBGRAPH_OUTPUT_DIR = os.path.join(WEBQSP_DIR, "dpr_outputs")

# output dir for top-2k relations + condensed relations + condensed hyper-relations
REL_CONDHYPERREL_SUBGRAPH_DIR = os.path.join(SUBGRAPH_OUTPUT_DIR, "dpr_top_2k_relations_per_question")
os.makedirs(REL_CONDHYPERREL_SUBGRAPH_DIR, exist_ok=True)

In [22]:
import pickle
import numpy as np
import faiss

vector_size = 768

def save_results_to_file(docids, scores, idxs, docid2rel, oufname):
    topk_rels = []
    for score, idx in zip(scores, idxs):
        topk_rels.append((docid2rel[docids[idx]], score))       
    # save topk (hyper) relations to file
    with open(oufname, 'wb') as ouf:
        pickle.dump(topk_rels, ouf)

            
# save rels + cond hyper rels
def save_topk_hyper_relations(question, topk):
    qvec = question['question']
    qid = question['qid']
    print("Processing", qid, "...")
    # save docid2rel map
    docid2rel = {}
    # relations
    tup = read_relations_for_question(qid)
    if tup is None:
        raise ValueError(f"unknown qid {qid}")
    else:
        _, relations = tup
    for i, rel in enumerate(relations):
        docid = f"{qid}.relations.{i}"
        docid2rel[docid] = (rel.subj, rel.rel, rel.obj)
        
    # condensed relations
    cond_relations = read_condensed_relations_for_question(qid)
    if cond_relations:
        for docid, rel in cond_relations:
            docid2rel[docid] = rel
    # condensed hyper relations
    hyper_relations = read_condensed_hyper_relations_for_question(qid)    
    if hyper_relations:
        for docid, rel in hyper_relations:
            docid2rel[docid] = rel
    
    index = faiss.IndexFlatIP(vector_size)
    docids = []
    # index relations
    if question['relations']:
        ids, vectors = zip(*question['relations'])
        docids += list(ids)
        vectors = np.array(vectors)
        assert len(docids) == len(vectors)
        index.add(vectors)

    # add condensed relations
    if question['condensed_relations']:
        ids, condensed_relations = zip(*question['condensed_relations'])
        docids += list(ids)
        if condensed_relations:
            index.add(np.array(condensed_relations))
    
    # add condensed hyper-relations
    if question['condensed_hyper_relations']:
        ids, condensed_hyper_relations = zip(*question['condensed_hyper_relations'])
        docids += list(ids)
        if condensed_hyper_relations:
            index.add(np.array(condensed_hyper_relations))
    
    if len(docids) == 0:
        # no relations for this question
        print("no relations for", qid)
        return
    scores, idxs = index.search(np.expand_dims(qvec[1], 0), min(topk, len(docids)))
    scores = scores[0]
    idxs = idxs[0]
    save_results_to_file(docids, scores, idxs, docid2rel, os.path.join(REL_CONDHYPERREL_SUBGRAPH_DIR, f"{qid}.pkl"))    

In [23]:
# read DPR embeddings for relations
# read and save one question after another to save memory
DPR_OUT_DIR = os.path.join(WEBQSP_DIR, "models", "dpr_predictions", "webqsp_relations")

NUM_SHARDS = 100
# 2000 relations should be more than enough to generate 100 150-token passages
topk = 2000

cur_qid = None
questions = {}

for shard in range(NUM_SHARDS):
    dpr_output_file = os.path.join(DPR_OUT_DIR, f"all_relations.dpr_out_{shard}")
    with open(dpr_output_file, 'rb') as inf:
        dpr_output = pickle.load(inf)
    for docid, vec in dpr_output:
        parts = docid.split('.')
        qid, dtype = parts[0], parts[1]
        if qid != cur_qid:
            # moving to a new question: save top-k
            if cur_qid is not None:
                # add condensed relations
                questions[cur_qid]['condensed_relations'] = condensed_rels_vectors[cur_qid]
                # add condensed hyper-relations
                questions[cur_qid]['condensed_hyper_relations'] = condensed_hyper_rels_vectors[cur_qid]
                save_topk_hyper_relations(questions[cur_qid], topk)
                # optional to save memory
                del questions[cur_qid]
            cur_qid = qid
        if qid not in questions:
            questions[qid] = {
                'qid': qid,
                'question': None,
                'relations': [],
            }
        question = questions[qid]
        if dtype == 'question':
            question[dtype] = (docid, vec)
        elif dtype == 'relations':
            question[dtype].append((docid, vec))
        else:
            raise ValueError(f"Unknown dtype {dtype}")

# save the last question
# add condensed relations
questions[cur_qid]['condensed_relations'] = condensed_rels_vectors[cur_qid]
# add condensed hyper-relations
questions[cur_qid]['condensed_hyper_relations'] = condensed_hyper_rels_vectors[cur_qid]

save_topk_hyper_relations(questions[cur_qid], topk)

Processing WebQTrn-0 ...
Processing WebQTrn-1 ...
Processing WebQTrn-3 ...
Processing WebQTrn-4 ...
Processing WebQTrn-5 ...
Processing WebQTrn-6 ...
Processing WebQTrn-7 ...
Processing WebQTrn-8 ...
Processing WebQTrn-9 ...
Processing WebQTrn-11 ...
Processing WebQTrn-12 ...
Processing WebQTrn-13 ...
Processing WebQTrn-14 ...
Processing WebQTrn-15 ...
Processing WebQTrn-17 ...
Processing WebQTrn-18 ...
Processing WebQTrn-20 ...
Processing WebQTrn-21 ...
Processing WebQTrn-22 ...
Processing WebQTrn-23 ...
Processing WebQTrn-24 ...
Processing WebQTrn-25 ...
Processing WebQTrn-26 ...
Processing WebQTrn-30 ...
Processing WebQTrn-31 ...
Processing WebQTrn-32 ...
Processing WebQTrn-33 ...
Processing WebQTrn-34 ...
Processing WebQTrn-35 ...
Processing WebQTrn-36 ...
Processing WebQTrn-37 ...
Processing WebQTrn-38 ...
Processing WebQTrn-40 ...
Processing WebQTrn-42 ...
Processing WebQTrn-44 ...
Processing WebQTrn-45 ...
Processing WebQTrn-46 ...
Processing WebQTrn-47 ...
Processing WebQTrn-49

Processing WebQTrn-379 ...
Processing WebQTrn-380 ...
Processing WebQTrn-381 ...
Processing WebQTrn-382 ...
Processing WebQTrn-383 ...
Processing WebQTrn-385 ...
Processing WebQTrn-386 ...
Processing WebQTrn-387 ...
Processing WebQTrn-388 ...
Processing WebQTrn-389 ...
Processing WebQTrn-390 ...
Processing WebQTrn-391 ...
Processing WebQTrn-393 ...
Processing WebQTrn-395 ...
Processing WebQTrn-397 ...
Processing WebQTrn-398 ...
Processing WebQTrn-399 ...
Processing WebQTrn-400 ...
Processing WebQTrn-401 ...
Processing WebQTrn-402 ...
Processing WebQTrn-403 ...
Processing WebQTrn-404 ...
Processing WebQTrn-405 ...
Processing WebQTrn-406 ...
Processing WebQTrn-407 ...
Processing WebQTrn-408 ...
Processing WebQTrn-409 ...
Processing WebQTrn-410 ...
Processing WebQTrn-411 ...
Processing WebQTrn-412 ...
Processing WebQTrn-413 ...
Processing WebQTrn-414 ...
Processing WebQTrn-415 ...
Processing WebQTrn-416 ...
Processing WebQTrn-417 ...
Processing WebQTrn-418 ...
Processing WebQTrn-420 ...
P

Processing WebQTrn-750 ...
Processing WebQTrn-751 ...
Processing WebQTrn-752 ...
Processing WebQTrn-753 ...
Processing WebQTrn-754 ...
Processing WebQTrn-756 ...
Processing WebQTrn-757 ...
Processing WebQTrn-758 ...
Processing WebQTrn-759 ...
Processing WebQTrn-761 ...
Processing WebQTrn-762 ...
Processing WebQTrn-764 ...
Processing WebQTrn-765 ...
Processing WebQTrn-766 ...
Processing WebQTrn-767 ...
Processing WebQTrn-768 ...
Processing WebQTrn-769 ...
Processing WebQTrn-770 ...
Processing WebQTrn-771 ...
Processing WebQTrn-772 ...
Processing WebQTrn-773 ...
Processing WebQTrn-775 ...
Processing WebQTrn-776 ...
Processing WebQTrn-780 ...
Processing WebQTrn-781 ...
Processing WebQTrn-782 ...
Processing WebQTrn-783 ...
Processing WebQTrn-784 ...
Processing WebQTrn-785 ...
Processing WebQTrn-786 ...
Processing WebQTrn-788 ...
Processing WebQTrn-790 ...
Processing WebQTrn-792 ...
Processing WebQTrn-793 ...
Processing WebQTrn-794 ...
Processing WebQTrn-795 ...
Processing WebQTrn-796 ...
P

Processing WebQTrn-1122 ...
Processing WebQTrn-1123 ...
Processing WebQTrn-1124 ...
Processing WebQTrn-1125 ...
Processing WebQTrn-1126 ...
Processing WebQTrn-1127 ...
Processing WebQTrn-1128 ...
Processing WebQTrn-1129 ...
Processing WebQTrn-1130 ...
Processing WebQTrn-1131 ...
Processing WebQTrn-1132 ...
Processing WebQTrn-1133 ...
Processing WebQTrn-1134 ...
Processing WebQTrn-1135 ...
Processing WebQTrn-1136 ...
Processing WebQTrn-1138 ...
Processing WebQTrn-1141 ...
Processing WebQTrn-1142 ...
Processing WebQTrn-1143 ...
Processing WebQTrn-1144 ...
Processing WebQTrn-1148 ...
Processing WebQTrn-1150 ...
Processing WebQTrn-1151 ...
Processing WebQTrn-1152 ...
Processing WebQTrn-1153 ...
Processing WebQTrn-1155 ...
Processing WebQTrn-1156 ...
Processing WebQTrn-1157 ...
Processing WebQTrn-1158 ...
Processing WebQTrn-1160 ...
Processing WebQTrn-1161 ...
Processing WebQTrn-1162 ...
Processing WebQTrn-1163 ...
Processing WebQTrn-1164 ...
Processing WebQTrn-1165 ...
Processing WebQTrn-1

Processing WebQTrn-1480 ...
Processing WebQTrn-1481 ...
Processing WebQTrn-1482 ...
Processing WebQTrn-1484 ...
Processing WebQTrn-1485 ...
Processing WebQTrn-1486 ...
Processing WebQTrn-1487 ...
Processing WebQTrn-1488 ...
Processing WebQTrn-1489 ...
Processing WebQTrn-1492 ...
Processing WebQTrn-1493 ...
Processing WebQTrn-1494 ...
Processing WebQTrn-1495 ...
Processing WebQTrn-1496 ...
Processing WebQTrn-1497 ...
Processing WebQTrn-1498 ...
Processing WebQTrn-1500 ...
Processing WebQTrn-1502 ...
Processing WebQTrn-1503 ...
Processing WebQTrn-1504 ...
Processing WebQTrn-1505 ...
Processing WebQTrn-1506 ...
Processing WebQTrn-1507 ...
Processing WebQTrn-1508 ...
Processing WebQTrn-1511 ...
Processing WebQTrn-1512 ...
Processing WebQTrn-1513 ...
Processing WebQTrn-1515 ...
Processing WebQTrn-1516 ...
Processing WebQTrn-1517 ...
Processing WebQTrn-1519 ...
Processing WebQTrn-1520 ...
Processing WebQTrn-1521 ...
Processing WebQTrn-1522 ...
Processing WebQTrn-1523 ...
Processing WebQTrn-1

...

Processing WebQTest-617 ...
Processing WebQTest-618 ...
Processing WebQTest-619 ...
Processing WebQTest-620 ...
Processing WebQTest-621 ...
Processing WebQTest-622 ...
Processing WebQTest-624 ...
Processing WebQTest-626 ...
Processing WebQTest-627 ...
Processing WebQTest-628 ...
Processing WebQTest-629 ...
Processing WebQTest-630 ...
Processing WebQTest-631 ...
Processing WebQTest-633 ...
Processing WebQTest-634 ...
Processing WebQTest-635 ...
Processing WebQTest-636 ...
Processing WebQTest-637 ...
Processing WebQTest-638 ...
Processing WebQTest-639 ...
Processing WebQTest-640 ...
Processing WebQTest-642 ...
Processing WebQTest-643 ...
Processing WebQTest-644 ...
Processing WebQTest-645 ...
Processing WebQTest-646 ...
Processing WebQTest-648 ...
Processing WebQTest-649 ...
Processing WebQTest-650 ...
Processing WebQTest-651 ...
Processing WebQTest-652 ...
Processing WebQTest-653 ...
Processing WebQTest-654 ...
Processing WebQTest-655 ...
Processing WebQTest-656 ...
Processing WebQTest-

Processing WebQTest-977 ...
Processing WebQTest-978 ...
Processing WebQTest-979 ...
Processing WebQTest-980 ...
Processing WebQTest-982 ...
Processing WebQTest-983 ...
Processing WebQTest-984 ...
Processing WebQTest-985 ...
Processing WebQTest-986 ...
Processing WebQTest-987 ...
Processing WebQTest-988 ...
Processing WebQTest-989 ...
Processing WebQTest-990 ...
Processing WebQTest-991 ...
Processing WebQTest-993 ...
Processing WebQTest-994 ...
Processing WebQTest-996 ...
Processing WebQTest-997 ...
Processing WebQTest-998 ...
Processing WebQTest-999 ...
Processing WebQTest-1000 ...
Processing WebQTest-1001 ...
Processing WebQTest-1002 ...
Processing WebQTest-1003 ...
Processing WebQTest-1004 ...
Processing WebQTest-1005 ...
Processing WebQTest-1006 ...
Processing WebQTest-1007 ...
Processing WebQTest-1008 ...
Processing WebQTest-1009 ...
Processing WebQTest-1010 ...
Processing WebQTest-1011 ...
Processing WebQTest-1012 ...
Processing WebQTest-1014 ...
Processing WebQTest-1015 ...
Proce

Processing WebQTest-1327 ...
Processing WebQTest-1328 ...
Processing WebQTest-1329 ...
Processing WebQTest-1330 ...
Processing WebQTest-1331 ...
Processing WebQTest-1332 ...
Processing WebQTest-1333 ...
Processing WebQTest-1334 ...
Processing WebQTest-1335 ...
Processing WebQTest-1336 ...
Processing WebQTest-1337 ...
Processing WebQTest-1339 ...
Processing WebQTest-1340 ...
Processing WebQTest-1341 ...
Processing WebQTest-1342 ...
Processing WebQTest-1343 ...
Processing WebQTest-1344 ...
Processing WebQTest-1348 ...
Processing WebQTest-1350 ...
Processing WebQTest-1351 ...
Processing WebQTest-1353 ...
Processing WebQTest-1354 ...
Processing WebQTest-1355 ...
Processing WebQTest-1356 ...
Processing WebQTest-1357 ...
Processing WebQTest-1358 ...
Processing WebQTest-1359 ...
Processing WebQTest-1361 ...
Processing WebQTest-1362 ...
Processing WebQTest-1363 ...
Processing WebQTest-1364 ...
Processing WebQTest-1365 ...
Processing WebQTest-1366 ...
Processing WebQTest-1367 ...
Processing Web

Processing WebQTest-1679 ...
Processing WebQTest-1680 ...
Processing WebQTest-1682 ...
Processing WebQTest-1683 ...
Processing WebQTest-1684 ...
Processing WebQTest-1685 ...
Processing WebQTest-1686 ...
Processing WebQTest-1687 ...
Processing WebQTest-1688 ...
Processing WebQTest-1690 ...
Processing WebQTest-1691 ...
Processing WebQTest-1694 ...
Processing WebQTest-1695 ...
Processing WebQTest-1696 ...
Processing WebQTest-1698 ...
Processing WebQTest-1699 ...
Processing WebQTest-1700 ...
Processing WebQTest-1701 ...
Processing WebQTest-1702 ...
Processing WebQTest-1703 ...
Processing WebQTest-1705 ...
Processing WebQTest-1706 ...
Processing WebQTest-1707 ...
Processing WebQTest-1708 ...
Processing WebQTest-1710 ...
Processing WebQTest-1711 ...
Processing WebQTest-1712 ...
Processing WebQTest-1713 ...
Processing WebQTest-1714 ...
Processing WebQTest-1715 ...
Processing WebQTest-1719 ...
Processing WebQTest-1721 ...
Processing WebQTest-1723 ...
Processing WebQTest-1724 ...
Processing Web

Processing WebQTest-2028 ...
Processing WebQTest-2029 ...
Processing WebQTest-2030 ...
Processing WebQTest-2031 ...


## FiD Input

Create FiD input for the DPR-pruned subgraphs

In [24]:
FID_INPUT_DIR = os.path.join(WEBQSP_DIR, "fid_inputs")

In [25]:
def convert_subgraph_to_passages(
    qid, subgraph, entity_names, MAX_TOKENS, MAX_PASSAGES, IGNORE_CVT
):
    passages = []
    tok_cnt = 0
    sentences = []
    seen_sentences = set()
    
    for relation, _ in subgraph:
        sentence = convert_hyper_relation_to_text(relation, entity_names, ignore_cvt=IGNORE_CVT)
        if sentence is None or sentence in seen_sentences:
            continue
        seen_sentences.add(sentence)
        num_tokens = len(sentence.split(' '))
        if num_tokens > MAX_TOKENS:
            if " person quotations " in sentence:
                # skip the quotation relations
                continue
            print("sentence too long!", sentence, num_tokens)
        if tok_cnt + num_tokens > MAX_TOKENS:
            # create a new passage
            passage = {
                "text": ' '.join(sentences),
                "title": '',
            }
            passages.append(passage)
            if len(passages) >= MAX_PASSAGES:
                # got enough passages
                break
            sentences = [sentence]
            tok_cnt = num_tokens
        else:
            sentences.append(sentence)
            tok_cnt += num_tokens
    if sentences:
        passage = {
            "text": ' '.join(sentences),
            "title": '',
        }
        passages.append(passage)

    if len(passages) == 0:
        passages.append({"text":'', "title":''})

    return passages

In [26]:
import json

def save_fid_input(
    pruned_graph_dir, 
    output_dir, 
    MAX_PASSAGES, 
    MAX_TOKENS, 
    IGNORE_CVT=False, 
    convert_subgraph_to_passages=convert_subgraph_to_passages
):
    results = defaultdict(list)
    for split in ['train', 'test']:
        for question in webqsp_questions[split]:
            out_split = split
            qid = question['QuestionId']
            # use WebQTrn-0 to WebQTrn-377 as dev set
            if qid.startswith("WebQTrn-"):
                num = int(qid[8:])
                if num <= 376:
                    out_split = 'dev'
            print("Processing", qid, "...")
            qtext = question['QuestionText']
            answers = [ans['text'] if ans['text'] is not None else ans['freebaseId'] for ans in question['Answers']]
            if not answers:
                answers = ["None"]

            pruned_graph_oufname = os.path.join(pruned_graph_dir, f"{qid}.pkl")
            if os.path.exists(pruned_graph_oufname):
                with open(pruned_graph_oufname, 'rb') as inf:
                    subgraph = pickle.load(inf)
                passages = convert_subgraph_to_passages(
                    qid, subgraph, entity_names, MAX_TOKENS, MAX_PASSAGES, IGNORE_CVT
                )
            else:
                passages = [{"text":'', "title":''}]
            
            results[out_split].append({
                "id": qid,
                "question": qtext,
                "answers": answers,
                "ctxs": passages,
            })

    for split, samples in results.items():
        oufname = os.path.join(output_dir, f"webqsp_{split}.json")
        with open(oufname, 'w') as ouf:
            json.dump(samples, ouf)

In [27]:
# DPR-ranked top relations to form 100 passages (max 150 tokens each)
# each passage contains relations ranked by DPR scores until it reaches max tokens

MAX_TOKENS = 150
MAX_PASSAGES = 100

In [28]:
# top 100 passages with relations + condensed hyper relations
FID_INPUT_TOP_COND_HYPER_RELS_DIR = os.path.join(FID_INPUT_DIR)
os.makedirs(FID_INPUT_TOP_COND_HYPER_RELS_DIR, exist_ok=True)

In [29]:

save_fid_input(
    REL_CONDHYPERREL_SUBGRAPH_DIR, 
    FID_INPUT_TOP_COND_HYPER_RELS_DIR, 
    MAX_PASSAGES=MAX_PASSAGES, 
    MAX_TOKENS=MAX_TOKENS,
    IGNORE_CVT=True,
)

Processing WebQTrn-0 ...
Processing WebQTrn-1 ...
Processing WebQTrn-3 ...
Processing WebQTrn-4 ...
Processing WebQTrn-5 ...
Processing WebQTrn-6 ...
Processing WebQTrn-7 ...
Processing WebQTrn-8 ...
Processing WebQTrn-9 ...
Processing WebQTrn-11 ...
Processing WebQTrn-12 ...
Processing WebQTrn-13 ...
Processing WebQTrn-14 ...
Processing WebQTrn-15 ...
Processing WebQTrn-17 ...
Processing WebQTrn-18 ...
Processing WebQTrn-20 ...
Processing WebQTrn-21 ...
Processing WebQTrn-22 ...
Processing WebQTrn-23 ...
Processing WebQTrn-24 ...
Processing WebQTrn-25 ...
Processing WebQTrn-26 ...
Processing WebQTrn-30 ...
Processing WebQTrn-31 ...
Processing WebQTrn-32 ...
Processing WebQTrn-33 ...
Processing WebQTrn-34 ...
Processing WebQTrn-35 ...
Processing WebQTrn-36 ...
Processing WebQTrn-37 ...
Processing WebQTrn-38 ...
Processing WebQTrn-40 ...
Processing WebQTrn-42 ...
Processing WebQTrn-44 ...
Processing WebQTrn-45 ...
Processing WebQTrn-46 ...
Processing WebQTrn-47 ...
Processing WebQTrn-49

Processing WebQTrn-78 ...
Processing WebQTrn-80 ...
Processing WebQTrn-83 ...
Processing WebQTrn-86 ...
Processing WebQTrn-87 ...
Processing WebQTrn-88 ...
Processing WebQTrn-90 ...
Processing WebQTrn-91 ...
Processing WebQTrn-92 ...
Processing WebQTrn-95 ...
Processing WebQTrn-96 ...
Processing WebQTrn-97 ...
Processing WebQTrn-98 ...
Processing WebQTrn-100 ...
Processing WebQTrn-101 ...
Processing WebQTrn-102 ...
Processing WebQTrn-103 ...
Processing WebQTrn-104 ...
Processing WebQTrn-105 ...
Processing WebQTrn-107 ...
Processing WebQTrn-108 ...
Processing WebQTrn-111 ...
Processing WebQTrn-112 ...
Processing WebQTrn-113 ...
Processing WebQTrn-114 ...
Processing WebQTrn-115 ...
Processing WebQTrn-116 ...
Processing WebQTrn-117 ...
Processing WebQTrn-118 ...
Processing WebQTrn-119 ...
Processing WebQTrn-120 ...
sentence too long! Family Guy - Season 2 regular tv appearance actor Alex Borstein , and regular tv appearance character Lois Griffin , and regular tv appearance from 1999-01-3

Processing WebQTrn-125 ...
Processing WebQTrn-126 ...
Processing WebQTrn-128 ...
Processing WebQTrn-129 ...
Processing WebQTrn-130 ...
Processing WebQTrn-131 ...
Processing WebQTrn-133 ...
Processing WebQTrn-134 ...
Processing WebQTrn-135 ...
Processing WebQTrn-136 ...
Processing WebQTrn-137 ...
Processing WebQTrn-138 ...
Processing WebQTrn-139 ...
Processing WebQTrn-141 ...
Processing WebQTrn-142 ...
Processing WebQTrn-143 ...
Processing WebQTrn-144 ...
Processing WebQTrn-145 ...
Processing WebQTrn-146 ...
Processing WebQTrn-147 ...
Processing WebQTrn-148 ...
Processing WebQTrn-150 ...
Processing WebQTrn-152 ...
Processing WebQTrn-153 ...
Processing WebQTrn-155 ...
Processing WebQTrn-157 ...
Processing WebQTrn-158 ...
Processing WebQTrn-159 ...
Processing WebQTrn-160 ...
Processing WebQTrn-161 ...
Processing WebQTrn-162 ...
Processing WebQTrn-163 ...
Processing WebQTrn-164 ...
Processing WebQTrn-165 ...
Processing WebQTrn-166 ...
Processing WebQTrn-167 ...
Processing WebQTrn-168 ...
P

Processing WebQTrn-358 ...
Processing WebQTrn-360 ...
Processing WebQTrn-361 ...
Processing WebQTrn-363 ...
Processing WebQTrn-366 ...
Processing WebQTrn-367 ...
Processing WebQTrn-368 ...
Processing WebQTrn-369 ...
Processing WebQTrn-370 ...
Processing WebQTrn-372 ...
Processing WebQTrn-373 ...
Processing WebQTrn-375 ...
Processing WebQTrn-376 ...
Processing WebQTrn-377 ...
Processing WebQTrn-378 ...
Processing WebQTrn-379 ...
Processing WebQTrn-380 ...
Processing WebQTrn-381 ...
Processing WebQTrn-382 ...
Processing WebQTrn-383 ...
Processing WebQTrn-385 ...
Processing WebQTrn-386 ...
Processing WebQTrn-387 ...
Processing WebQTrn-388 ...
Processing WebQTrn-389 ...
Processing WebQTrn-390 ...
Processing WebQTrn-391 ...
Processing WebQTrn-393 ...
Processing WebQTrn-395 ...
Processing WebQTrn-397 ...
Processing WebQTrn-398 ...
Processing WebQTrn-399 ...
Processing WebQTrn-400 ...
Processing WebQTrn-401 ...
Processing WebQTrn-402 ...
Processing WebQTrn-403 ...
Processing WebQTrn-404 ...
P

Processing WebQTrn-606 ...
Processing WebQTrn-607 ...
Processing WebQTrn-608 ...
Processing WebQTrn-610 ...
Processing WebQTrn-611 ...
Processing WebQTrn-612 ...
Processing WebQTrn-613 ...
Processing WebQTrn-614 ...
Processing WebQTrn-615 ...
Processing WebQTrn-616 ...
Processing WebQTrn-617 ...
Processing WebQTrn-618 ...
Processing WebQTrn-619 ...
Processing WebQTrn-620 ...
Processing WebQTrn-621 ...
Processing WebQTrn-622 ...
Processing WebQTrn-624 ...
Processing WebQTrn-625 ...
Processing WebQTrn-627 ...
Processing WebQTrn-628 ...
Processing WebQTrn-629 ...
Processing WebQTrn-630 ...
Processing WebQTrn-631 ...
Processing WebQTrn-632 ...
sentence too long! Saturday Night Live award nomination award nominee Al Franken , and award nomination award nominee David Spade , and award nomination award nominee Robert Smigel , and award nomination award nominee Jack Handey , and award nomination award nominee Christine Zander , and award nomination award nominee Tim Meadows , and award nominat

...

Processing WebQTest-1107 ...
Processing WebQTest-1108 ...
Processing WebQTest-1109 ...
Processing WebQTest-1110 ...
Processing WebQTest-1111 ...
Processing WebQTest-1113 ...
Processing WebQTest-1114 ...
Processing WebQTest-1115 ...
Processing WebQTest-1116 ...
Processing WebQTest-1117 ...
Processing WebQTest-1118 ...
Processing WebQTest-1119 ...
Processing WebQTest-1120 ...
Processing WebQTest-1121 ...
Processing WebQTest-1124 ...
Processing WebQTest-1125 ...
Processing WebQTest-1126 ...
Processing WebQTest-1127 ...
Processing WebQTest-1130 ...
Processing WebQTest-1131 ...
Processing WebQTest-1132 ...
Processing WebQTest-1133 ...
Processing WebQTest-1135 ...
Processing WebQTest-1136 ...
Processing WebQTest-1137 ...
Processing WebQTest-1138 ...
Processing WebQTest-1142 ...
Processing WebQTest-1143 ...
Processing WebQTest-1144 ...
Processing WebQTest-1145 ...
Processing WebQTest-1146 ...
Processing WebQTest-1147 ...
Processing WebQTest-1148 ...
Processing WebQTest-1149 ...
Processing Web

Processing WebQTest-1402 ...
Processing WebQTest-1403 ...
Processing WebQTest-1405 ...
Processing WebQTest-1408 ...
Processing WebQTest-1409 ...
Processing WebQTest-1410 ...
Processing WebQTest-1414 ...
Processing WebQTest-1416 ...
Processing WebQTest-1417 ...
Processing WebQTest-1418 ...
Processing WebQTest-1419 ...
Processing WebQTest-1420 ...
Processing WebQTest-1421 ...
sentence too long! Woody Harrelson award honor award Satellite Special Achievement Award for Outstanding Motion Picture Ensemble , and award honor award winner George Clooney , and award honor award winner Adrien Brody , and award honor award winner John Cusack , and award honor award winner Dash Mihok , and award honor award winner Miranda Otto , and award honor award winner Jared Leto , and award honor award winner Jim Caviezel , and award honor award winner John C. Reilly , and award honor award winner John Savage , and award honor award winner Thomas Jane , and award honor award winner Kirk Acevedo , and award h

Processing WebQTest-1527 ...
Processing WebQTest-1528 ...
Processing WebQTest-1529 ...
Processing WebQTest-1531 ...
Processing WebQTest-1533 ...
Processing WebQTest-1534 ...
Processing WebQTest-1535 ...
Processing WebQTest-1536 ...
Processing WebQTest-1537 ...
Processing WebQTest-1538 ...
Processing WebQTest-1539 ...
Processing WebQTest-1540 ...
Processing WebQTest-1541 ...
Processing WebQTest-1542 ...
Processing WebQTest-1543 ...
Processing WebQTest-1544 ...
Processing WebQTest-1546 ...
Processing WebQTest-1547 ...
Processing WebQTest-1548 ...
Processing WebQTest-1550 ...
Processing WebQTest-1551 ...
Processing WebQTest-1553 ...
Processing WebQTest-1554 ...
Processing WebQTest-1555 ...
Processing WebQTest-1556 ...
Processing WebQTest-1557 ...
Processing WebQTest-1558 ...
Processing WebQTest-1559 ...
Processing WebQTest-1560 ...
Processing WebQTest-1561 ...
Processing WebQTest-1562 ...
Processing WebQTest-1563 ...
Processing WebQTest-1564 ...
Processing WebQTest-1565 ...
Processing Web

Processing WebQTest-1744 ...
Processing WebQTest-1746 ...
Processing WebQTest-1748 ...
Processing WebQTest-1750 ...
Processing WebQTest-1751 ...
Processing WebQTest-1752 ...
Processing WebQTest-1753 ...
Processing WebQTest-1754 ...
Processing WebQTest-1755 ...
Processing WebQTest-1756 ...
Processing WebQTest-1757 ...
Processing WebQTest-1758 ...
Processing WebQTest-1759 ...
Processing WebQTest-1760 ...
Processing WebQTest-1761 ...
Processing WebQTest-1763 ...
Processing WebQTest-1764 ...
Processing WebQTest-1765 ...
Processing WebQTest-1766 ...
Processing WebQTest-1767 ...
Processing WebQTest-1768 ...
Processing WebQTest-1769 ...
Processing WebQTest-1770 ...
Processing WebQTest-1771 ...
Processing WebQTest-1772 ...
Processing WebQTest-1773 ...
Processing WebQTest-1774 ...
Processing WebQTest-1775 ...
Processing WebQTest-1776 ...
Processing WebQTest-1777 ...
Processing WebQTest-1779 ...
Processing WebQTest-1780 ...
Processing WebQTest-1781 ...
Processing WebQTest-1782 ...
Processing Web

Processing WebQTest-1916 ...
Processing WebQTest-1917 ...
Processing WebQTest-1919 ...
Processing WebQTest-1920 ...
Processing WebQTest-1921 ...
Processing WebQTest-1922 ...
Processing WebQTest-1923 ...
Processing WebQTest-1925 ...
Processing WebQTest-1926 ...
Processing WebQTest-1928 ...
Processing WebQTest-1929 ...
Processing WebQTest-1932 ...
Processing WebQTest-1933 ...
Processing WebQTest-1934 ...
Processing WebQTest-1935 ...
Processing WebQTest-1936 ...
Processing WebQTest-1937 ...
Processing WebQTest-1938 ...
Processing WebQTest-1939 ...
Processing WebQTest-1940 ...
Processing WebQTest-1941 ...
Processing WebQTest-1942 ...
Processing WebQTest-1943 ...
Processing WebQTest-1944 ...
Processing WebQTest-1946 ...
Processing WebQTest-1947 ...
Processing WebQTest-1950 ...
Processing WebQTest-1952 ...
Processing WebQTest-1954 ...
Processing WebQTest-1955 ...
Processing WebQTest-1956 ...
Processing WebQTest-1957 ...
Processing WebQTest-1958 ...
Processing WebQTest-1960 ...
Processing Web

In [30]:
# Answer Coverage
# check retrieval accuracy at top 10 and top 100
from collections import Counter

def ans_in_ctxs(answers, ctxs, topk=0):
    if topk <= 0:
        topk = len(ctxs)
    for ans in answers:
        for ctx in ctxs[:topk]:
            if ans in ctx['text']:
                return True
    return False


def get_ans_recall(fid_input_dir):
    topks = [10, 100]
    for split in ['train', 'dev', 'test']:
        infname = os.path.join(fid_input_dir, f"webqsp_{split}.json")
        with open(infname) as inf:
            samples = json.load(inf)
        hit = Counter()
        ttl = 0
        for question in samples:
            ttl += 1
            for topk in topks:
                if ans_in_ctxs(question['answers'], question['ctxs'], topk):
                    hit[topk] += 1
        print("Split:", split)
        for topk in topks:
            print(f"Top {topk} passages, answer found in {hit[topk]}/{ttl} (= {hit[topk]/ttl}) questions")

In [32]:
# The top-100 answer coverage is around 95-96%
get_ans_recall(FID_INPUT_TOP_COND_HYPER_RELS_DIR)

Split: train
Top 10 passages, answer found in 2451/2792 (= 0.877865329512894) questions
Top 100 passages, answer found in 2648/2792 (= 0.9484240687679083) questions
Split: dev
Top 10 passages, answer found in 263/306 (= 0.8594771241830066) questions
Top 100 passages, answer found in 292/306 (= 0.954248366013072) questions
Split: test
Top 10 passages, answer found in 1435/1639 (= 0.8755338621110433) questions
Top 100 passages, answer found in 1571/1639 (= 0.9585112873703477) questions


## Train FiD

Using the FiD input generated from the previous step, a FiD model can now be trained using the official FiD code (https://github.com/facebookresearch/FiD).

We release our T5-large FiD checkpoint [here](https://dl.fbaipublicfiles.com/UniK-QA/fid_checkpoint.tar.xz).

The evaluation results of this checkpoint should look something like:

[12/10/2020 14:31:55] {test.py:65} WARNING - 0, total 1639 -- average = 0.791<br>
[12/10/2020 14:31:55] {test.py:72} INFO - total number of example 1639<br>
[12/10/2020 14:31:56] {test.py:132} INFO - EM 0.791336